In [ ]:
# Import Space
import cv2
import sys
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal
import time
import math
import imageio
import glob, os
import datetime
import heapq
import numpy as np
from scipy.signal import find_peaks_cwt
from matplotlib.patches import Ellipse


def smooth(x, window_len=11, window='hanning'):
    """
    Acceptable inputs for window variable:
        'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays.")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size.")

    if window_len < 3:
        return x

    if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
        raise ValueError(
            "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        )

    s = np.r_[x[window_len - 1:0:-1], x, x[-2:-window_len - 1:-1]]
    
    if window == 'flat':  #moving average
        w = np.ones(window_len, 'd')
    else:
        w = eval('np.' + window + '(window_len)')

    y = np.convolve(w / w.sum(), s, mode='valid')
    return y


# Return vector with zeroes in place of of values outside min/max bounds
def notchFilter(inputVec, minVec, maxVec=sys.maxsize):
    returnVec = np.copy(inputVec)
    for i in range(inputVec.size):
        if inputVec[i] < minVec or inputVec[i] > maxVec:
            returnVec[i] = 0
    return returnVec


# Count number of nonzero intervals in signal
def peakNumber(cleanVector, minZeroes=1):
    numPeaks = 0
    count = 0
    peak = True
    for datapoint in cleanVector:
        if (datapoint > 0 and peak):
            print(datapoint)
            numPeaks += 1
            peak = False
            count = 0
        else:
            if (datapoint == 0):
                count += 1
            if (count >= minZeroes):
                peak = True
    return numPeaks


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = scipy.signal.butter(
        order, normal_cutoff, btype='high', analog=False)
    return b, a

# Attenuate low frequencies, pass high.
def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = scipy.signal.filtfilt(b, a, data)
    return y


def butter_lowpass(cutOff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutOff / nyq
    b, a = scipy.signal.butter(order, normalCutoff, btype='low', analog=True)
    return b, a

# Attenuate high frequencies, pass low.
def butter_lowpass_filter(data, cutOff, fs, order=4):
    b, a = butter_lowpass(cutOff, fs, order=order)
    y = scipy.signal.lfilter(b, a, data)
    return y

# Find aTan2 of y,x vector. Returns degrees within 0-360.
def angle(y, x):
    return ((57.2958 * math.atan2(y, x) + 360) % 360)


In [ ]:
# Return normalized cut of inputVector
def vectorWindow(inputVector, prev, future, keyPoint):
    outputVector = inputVector.copy()
    dif1 = keyPoint+future-outputVector.size
    dif2 = keyPoint-prev 
    
    # Size compensation 
    if(dif1 > 0):
        #print("long")
        key = outputVector[-1]
        outputVector = np.append(outputVector, np.array([key for i in range(dif1)]))
    if(dif2 < 0):
        #print("short")
        key = outputVector[0]
        outputVector = np.append(np.array([key for i in range(abs(dif2))]), outputVector)
        keyPoint += abs(dif2)
        
    outputVector = outputVector[keyPoint-prev:keyPoint+future]
    outputVector = outputVector/np.linalg.norm(outputVector)
    outputVector = smooth(outputVector, window_len=1)
    return outputVector

def gradientWindow(inputVector, gradientVector, prev, future):
    # Find peak of gradient vector
    keyPoint = int(np.argmax(gradientVector)/2)
    return vectorWindow(inputVector, prev, future, keyPoint)

# Shift vector mean down to zero
def zeroMean(inputVector):
    outputVector = inputVector.copy()
    mu = np.average(inputVector)
    for i in range(len(outputVector)):
        outputVector[i] = outputVector[i] - mu
    return outputVector

# Scales vector linearly by column
def scale_linear_bycolumn(rawpoints, high=100.0, low=0.0):
    mins = np.min(rawpoints, axis=0)
    maxs = np.max(rawpoints, axis=0)
    rng = maxs - mins
    return high - (((high - low) * (maxs - rawpoints)) / rng)

In [ ]:
# Load all the data into arrays for more convenient access
d = [np.load("7distance.npy"), np.load("8distance.npy"), np.load("9distance.npy"), np.load("10distance.npy"), np.load("11distance.npy"), np.load("12distance.npy"), np.load("13distance.npy"), np.load("14distance.npy"), np.load("15distance.npy"), np.load("16distance.npy"), np.load("17distance.npy"), np.load("18distance.npy"), np.load("19distance.npy")]
g = [np.load("7gradient.npy"), np.load("8gradient.npy"), np.load("9gradient.npy"), np.load("10gradient.npy"), np.load("11gradient.npy"), np.load("12gradient.npy"), np.load("13gradient.npy"), np.load("14gradient.npy"), np.load("15gradient.npy"), np.load("16gradient.npy"), np.load("17gradient.npy"), np.load("18gradient.npy"), np.load("19gradient.npy")]
a = [np.load("7angle.npy"), np.load("8angle.npy"), np.load("9angle.npy"), np.load("10angle.npy"), np.load("11angle.npy"), np.load("12angle.npy"), np.load("13angle.npy"), np.load("14angle.npy"), np.load("15angle.npy"), np.load("16angle.npy"), np.load("17angle.npy"), np.load("18angle.npy"), np.load("19angle.npy")]



# Initialize final arrays
dis = []
angs = []

# Set width of window
past = 300
future = 300

for i in range(len(d)):
    dis.append(gradientWindow(d[i], g[i], past, future))
    angs.append(gradientWindow(a[i], g[i], past, future))

    
# Average and scale functions for plotting    
averageDis = np.sum(np.array(np.array(dis)), axis=0)
averageDisP = scale_linear_bycolumn((averageDis/np.linalg.norm(averageDis)), 1,  0)
averageAng = np.sum(np.array(np.array(angs)), axis=0)
averageAngP = scale_linear_bycolumn((averageAng/np.linalg.norm(averageAng)), 1, 0)

# Index for plotting
index = np.linspace(-past/120, future/120, len(averageDisP))

# ANGLE
plt.plot(index, averageDisP, color='g', label="distance")
plt.plot(index, averageAngP, color='b', label="angular difference")
plt.legend(bbox_to_anchor=(1,1),loc=4,borderaxespad=0.,fontsize='small')
# Best Fit
plt.plot(index, np.poly1d(np.polyfit(index, averageDisP, 3))(np.unique(index)), c=(0,1,0.15), linestyle='--', alpha=0.5)
plt.plot(index, np.poly1d(np.polyfit(index, averageAngP, 3))(np.unique(index)), c=(0,0.15,1), linestyle='--', alpha=0.5)

plt.ylabel("Normalized Scale")
plt.xlabel("Time (s)")
plt.axis([-past/120, future/120, 0, 1.1])
plt.show()

print(np.correlate(averageAngP/np.linalg.norm(averageAngP), averageDisP/np.linalg.norm(averageDisP)))